In [40]:
import numpy as np
import matplotlib.pyplot as plt 
import uproot
import glob, os
import sys
import re

# Code for reading files in order
numbers = re.compile(r'(\d+)')
def NumericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

def ReadTheoreticalData(material):
    filename = "{0}/stoppingPowerData/format_p_{1}_SRIM.txt".format(os.getcwd(),material)
    file = np.loadtxt(filename, skiprows=1)
    energy = file[:,0]  # mg to g/
    dEdxElec = file[:,1]/ 1e-03
    dEdxNuc = file[:,2]/ 1e-03
    dEdx = np.add(file[:,1],file[:,2]) / 1e-03 # Get sum of colluns 1 e 2 (of dE/dx)   
    return energy, dEdx, dEdxElec, dEdxNuc

currentPath = os.getcwd()

materials = ["Si", "SiC", "C"]

# start here figure to N e-/h 
fig1, ax1 = plt.subplots()
ax1.set_title(r'$e^{-}/h$ Generation')
ax1.set_ylabel(r'N $e^{-}/h$ pairs')
ax1.set_xlabel(r'Primary Energy (MeV)')
ax1.set_xscale('log')
ax1.set_yscale('log')
for material in materials:
    eDepMean = []
    eDepStd = []
    nElectronHoleMean = []
    nElectronHoleStd = []
    primaryEnergy = []
    energy, dEdx, dEdxElec, dEdxNuc = ReadTheoreticalData(material)
    fig2,ax2 = plt.subplots()
    ax2.set_title('{0} Stopping Power'.format(material))
    ax2.set_ylabel(r'$dE/dx$ (MeV/g/cm2)')
    ax2.set_xlabel(r'Primary Energy (MeV)')
    ax2.set_xlim(xmin=10, xmax=6e3)
    ax2.set_xscale('log')
    ax2.set_yscale('log')
    plt.plot(energy, dEdx, label="{0} - SRIM 2013".format(material))
    for filepath in sorted(glob.iglob("{0}/results/{1}/*".format(currentPath, material)), key=NumericalSort):
        file = uproot.open("{0}:Event".format(filepath))
        eDepMean.append(file["fEdep"].array(library="np").mean())
        eDepStd.append(file["fEdep"].array(library="np").std())
        nElectronHoleMean.append(file["fNumberEH"].array(library="np").mean())
        nElectronHoleStd.append(file["fNumberEH"].array(library="np").std())
        primaryEnergy.append(file["fPrimaryEnergy"].array(library="np").mean())
        #print(file["fPrimaryEnergy"].array(library="np").std())
    ax1.scatter(primaryEnergy,nElectronHoleMean, label="{0}".format(material))
    ax2.errorbar(primaryEnergy, eDepMean, yerr=eDepStd, fmt='o', label="{0} - Geant4 11.1.0".format(material))
    ax2.legend()
    ax2.figure.savefig("{0}-Validation.pdf".format(material))
    plt.close(fig2)

ax1.legend()
ax1.figure.savefig("EHgen.pdf")
plt.close(fig1)
    